In [ ]:
#see load_label_png.py also.
import os
import numpy as np
import PIL.Image

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = '/scratch/ssd/gallowaa/cciw/VOCdevkit/VOC2012_v024/'

label_path = os.path.join(path, 'SegmentationClass')
image_path = os.path.join(path, 'JPEGImages')

In [ ]:
w = 242 # pixels 

# Save cropped masks

In [ ]:
list_of_labels = os.listdir(label_path)
for i in range(len(list_of_labels)):
    lab_file = os.path.join(
        label_path, list_of_labels[i].split('.')[0])
    lab = PIL.Image.open(lab_file + '.png')

    cnt = 0
    for row in range(lab.size[0] // w):
        for col in range(lab.size[0] // w):
            top = w * row
            bot = w * (row + 1)
            left  = w * col
            right = w * (col + 1)
            lab.crop(
                (left, top, right, bot)
            ).save(lab_file + '-' + str(cnt) + '.png')
            cnt += 1
    print(i, 'finished saving %d images' % cnt)

# Save cropped images

In [ ]:
#img = PIL.Image.open(img_file)
#print(img.size)

In [ ]:
list_of_images = os.listdir(image_path)
for i in range(len(list_of_images)):
    img_file = os.path.join(
        image_path, list_of_images[i].split('.')[0])
    img = PIL.Image.open(img_file + '.jpg')

    cnt = 0
    for row in range(img.size[0] // w):
        for col in range(img.size[0] // w):
            top = w * row
            bot = w * (row + 1)
            left  = w * col
            right = w * (col + 1)
            img.crop(
                (left, top, right, bot)
            ).save(img_file + '-' + str(cnt) + '.jpg')
            cnt += 1
    print(i, 'finished saving %d images' % cnt)

# Visualization

In [ ]:
wvz = 242 # 340, 283, 242, 212
lbl = np.asarray(img)
n_plot = lbl.shape[0] // wvz
fig, axes = plt.subplots(n_plot, n_plot, figsize=(12, 12))

for row in range(lbl.shape[0] // wvz):
    for col in range(lbl.shape[0] // wvz):
        axes[row, col].imshow(lbl[row*wvz:(1+row)*wvz, col*wvz:(col+1)*wvz])
        axes[row, col].axis('off')
#plt.tight_layout()        
#plt.savefig(file + '_viz.png')

In [ ]:
'''
PIL.Image.fromarray(
    np.uint8(
        lbl[row*w:(1+row)*w, col*w:(col+1)*w] * 255
    )
).save(label_path + '-' + str(cnt) + '.png')
'''